In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action = 'ignore')
import keras

import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score 

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


In [3]:
data = pd.read_csv('data_preprocessing_3.csv')
data.head()

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,AGE,CSTMR_CNT,CNT,AMT,region_diff
0,2019,1,강원,강릉시,건강보조식품 소매업,1,20,4,4,311200,0
1,2019,1,강원,강릉시,건강보조식품 소매업,1,30,7,8,1374500,0
2,2019,1,강원,강릉시,건강보조식품 소매업,1,40,7,8,2764300,0
3,2019,1,강원,강릉시,건강보조식품 소매업,1,50,21,21,3690830,0
4,2019,1,강원,강릉시,건강보조식품 소매업,1,60,19,20,1433500,0


### 원핫인코딩

In [4]:
df1 = pd.get_dummies(data, columns = ['CARD_CCG_NM', 'SEX', 'region_diff','AGE'])

In [5]:
df1.head()

,year,month,CARD_SIDO_NM,STD_CLSS_NM,CSTMR_CNT,CNT,AMT,CARD_CCG_NM_가평군,CARD_CCG_NM_강남구,CARD_CCG_NM_강동구,...,region_diff_0,region_diff_1,region_diff_2,AGE_10,AGE_20,AGE_30,AGE_40,AGE_50,AGE_60,AGE_70
0,2019,1,강원,건강보조식품 소매업,4,4,311200,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,2019,1,강원,건강보조식품 소매업,7,8,1374500,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,2019,1,강원,건강보조식품 소매업,7,8,2764300,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,2019,1,강원,건강보조식품 소매업,21,21,3690830,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,2019,1,강원,건강보조식품 소매업,19,20,1433500,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [ ]:
df1_2019 = df1[df1['year']==2019].reset_index(drop=True)
df1_2020 = df1[df1['year']==2020].reset_index(drop=True)

In [ ]:
df1_2020.head()

In [ ]:
df1.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'], axis = 1)

In [ ]:
df1_2019.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'], axis = 1)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [7]:
sido_list = df1['CARD_SIDO_NM'].unique()
std_list = df1['STD_CLSS_NM'].unique()

len(sido_list)
len(std_list)

41

In [8]:
X = df1.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'], axis = 1)
y = df1['AMT']
print(X.shape,y.shape)

(2691685, 243) (2691685,)


In [17]:
mean_squared_error(y_test, y_pred)

115733092153929.75

In [9]:

# 그리드서치 파라미터 설정

# rf_grid = {'n_estimators' : [100, 300, 500, 800, 1200],
#     'max_depth' : [5, 8, 15, 25, 30],
#     'min_samples_split' : [2, 5, 10, 15, 100], 
#     'min_samples_leaf' : [1, 2, 5, 10]        
# }



param_grid = { "n_estimators"      : [250, 300,500,800,1200],
                       "max_features"      : [3, 5],
                       "max_depth"         : [10, 20,25,30],
                       "min_samples_split" : [2, 4,10,15,100] ,
                       "bootstrap": [True, False]}




# hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
#               min_samples_split = min_samples_split, 
#              min_samples_leaf = min_samples_leaf)

In [ ]:
for sido in sido_list:
    for std in std_list:
       
        df2 = df1[(df1['CARD_SIDO_NM'] == sido) & (df1['STD_CLSS_NM'] == std)]   
        
        if df2.shape[0] < 10:
            print("이상한 것 :", df2.shape[0])
            continue
            
        X = df2.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'], axis = 1)
        y = df2['AMT']
        
        
        X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=777)

        
        # 랜덤포레스트
        #forest = RandomForestRegressor(n_estimators=400, random_state=42, max_depth=30,max_features = 'auto',
                                      #criterion='mse',verbose=0)
        #forest.fit(X_train, y_train)
        
        
        
        # 그리드 서치
        
        
        
        model = RandomForestRegressor(random_state=30)
        
        grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=2)
        grid_search.fit(X, y)
        #print grid_search.best_params_
        
        
        
        
        #gird_search = GridSearchCV(forest,grid=rf_grid,scoring='accuracy',verbose=1,fraction_limit = 0.1)
        #gird_search.fit(X_train,y_train)
        
        print(sido, std)
        print('최고의 정확도: {0:.4f}'.format(grid_search.best_score_))
        print('최고의 파라미터: ',grid_search.best_params_)
        
        
        
       
        
        # 예측
        y_pred = grid_search.predict(X_test)
        print(mean_squared_error(y_test, y_pred))
        

강원 건강보조식품 소매업
최고의 정확도: 0.7622
최고의 파라미터:  {'bootstrap': True, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 4, 'n_estimators': 800}
143663307317.88434
강원 골프장 운영업
최고의 정확도: 0.9520
최고의 파라미터:  {'bootstrap': False, 'max_depth': 25, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 300}
6248864767.204252
강원 과실 및 채소 소매업
최고의 정확도: 0.9311
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
26585351.498052817
강원 관광 민예품 및 선물용품 소매업
최고의 정확도: 0.7251
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
595.6757548182318
강원 그외 기타 스포츠시설 운영업
최고의 정확도: 0.2869
최고의 파라미터:  {'bootstrap': True, 'max_depth': 10, 'max_features': 3, 'min_samples_split': 15, 'n_estimators': 500}
87979821706.36562
강원 그외 기타 종합 소매업
최고의 정확도: 0.9747
최고의 파라미터:  {'bootstrap': True, 'max_depth': 20, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 500}
406906433918.6687
강원 기타 대형 종합 소매업
최고의 

1390799300647.1362
경기 서양식 음식점업
최고의 정확도: 0.9813
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
4207026950136.64
경기 수산물 소매업
최고의 정확도: 0.9246
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 800}
404949677472.61163
경기 슈퍼마켓
최고의 정확도: 0.9872
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 800}
104202149588443.38
경기 스포츠 및 레크레이션 용품 임대업
최고의 정확도: 0.9199
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
3287595453802.153
경기 여관업
최고의 정확도: 0.9599
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
381031451287.9826
경기 욕탕업
최고의 정확도: 0.9440
최고의 파라미터:  {'bootstrap': False, 'max_depth': 30, 'max_features': 5, 'min_samples_split': 2, 'n_estimators': 1200}
112327282150.03372
경기 육류 소매업
최고의 정확도: 0.9442


In [50]:
df1[(df1['CARD_SIDO_NM'] == sido) & (df1['STD_CLSS_NM'] == std)].shape

(1, 246)

In [ ]:
gird_search = GridSearchCV(forest,grid=rf_grid,scoring='accuracy',verbose=1)
gird_search.fit(X_train,y_train)

In [ ]:
print('최고의 정확도: {0:.4f}'.format(gird_search.best_score_))
print('최고의 파라미터: ',gird_search.best_params_)

In [ ]:
model = RandomForestRegressor(random_state=30)
param_grid = { "n_estimators"      : [250, 300],
              "criterion"         : ["gini", "entropy"],
              "max_features"      : [3, 5],
              "max_depth"         : [10, 20],
              "min_samples_split" : [2, 4] ,
              "bootstrap": [True, False]}
grid_search = GridSearchCV(clf, param_grid, n_jobs=-1, cv=2)
grid_search.fit(X, y)
print grid_search.best_params_

In [ ]:
# 랩덤포레스트 파라미터 : https://injo.tistory.com/30
# https://ssoondata.tistory.com/14
https://towardsdatascience.com/optimizing-hyperparameters-in-random-forest-classification-ec7741f9d3f6

In [ ]:
https://datamod.tistory.com/163
https://tensorflow.blog/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2-3-6-%EA%B2%B0%EC%A0%95-%ED%8A%B8%EB%A6%AC%EC%9D%98-%EC%95%99%EC%83%81%EB%B8%94/

In [ ]:
https://data-newbie.tistory.com/20


중요 매개변수 n_estimators , max_features max_depth 사전 가지치기 옵션이 있다

n_estimators 클수록 좋다 -> 많은 트리 -> 과적합 줄여준다. -> 긴 시간 

max_features 트리를 얼마나 무작위로 할지 -> 과대 적합 줄여준다. 

분류 :: max_features =sqrt(n_features)  RandomForestClassifier

회귀 : max_features = n_features   RandomForest Regressor

In [ ]:
밍슈가 보내준거 보기
https://www.kaggle.com/zelongq/simple-mlp-for-titanic-survival
https://programmers.co.kr/learn/courses/21/lessons/944

In [ ]:
# 타이타닉
https://www.kangtaeho.com/97
https://musma.github.io/2019/04/12/make-a-prediction2.html

In [ ]:
# 그리드 서치
https://lsjsj92.tistory.com/542

In [ ]:
# 랜덤포레스트회귀
https://wikidocs.net/26294
https://blog.naver.com/navdps/220621098783
    
    
# 파라미터 지정하는 방버
https://dive-into-ds.tistory.com/9    